Import Package

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q scann
!pip install tflite-support

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 65.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.19

In [ ]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
print(tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow import keras
import tensorflow_recommenders as tfrs
import os
import tempfile

2.11.1


Call Dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data capstone test coba revised 230610 (1).csv to data capstone test coba revised 230610 (1).csv


In [ ]:
dataset = pd.read_csv("Dataset Jejaknesia.csv")

In [ ]:
dataset.head()

,place_id,place_name,city,address,Lon,Lat,desc_place,price,category_1,category_2,category_3,combine_category,rating
0,1,Benteng Rotterdam,Sulawesi Selatan,"Bulo Gading, Kec. Ujung Pandang, Kota Makassar,",119.405598,-5.135639,Fort Rotterdam atau Benteng Ujung Pandang adal...,-,Seni & Budaya,Tempat Bersejarah,Museum,"Seni & Budaya,Tempat Bersejarah,Museum",3.5
1,2,Taman Nasional Lore Lindu,Sulawesi Tengah,"Dodolo, Sulawesi Tengah",120.239734,-1.473305,Taman Nasional Lore Lindu merupakan salah satu...,10000,Ruang Terbuka,Relaxing,Gunung,"Ruang Terbuka,Relaxing,Gunung,Hutan / Flora,Ma...",3.6
2,3,Tana Toraja,Sulawesi Selatan,"Lemo, Kec. Makale Utara, Kab. Tana Toraja",119.887305,-3.048921,Lokasi ini terkenal akan budaya uniknya yang b...,-,Seni & Budaya,Tempat Bersejarah,Spiritual / Religi,"Seni & Budaya,Tempat Bersejarah,Spiritual / Re...",4.0
3,4,Taman Laut Bunaken,Sulawesi Utara,Sulawesi Utara,123.975002,0.624693,Taman Nasional Bunaken adalah taman laut yang ...,-,Hutan / Flora,Margasatwa / Fauna,Ruang Terbuka,"Hutan / Flora,Margasatwa / Fauna,Ruang Terbuka...",4.0
4,5,Wakatobi,Sulawesi Tenggara,Sulawesi Tenggara,120.527900,-1.847900,Taman Nasional Kepulauan Wakatobi di kabupaten...,-,Hutan / Flora,Margasatwa / Fauna,Ruang Terbuka,"Hutan / Flora,Margasatwa / Fauna,Ruang Terbuka...",4.3


In [ ]:
data = dataset[['place_id', 'place_name','city', 'category_1', 'category_2', 'category_3','rating']]

In [ ]:
data.isnull().sum()

place_id      0
place_name    0
city          0
category_1    0
category_2    0
category_3    0
rating        0
dtype: int64

In [ ]:
missing_rows = data.isna().any(axis=1)
print(data[missing_rows].index)

Int64Index([], dtype='int64')


In [ ]:
shape = data.shape
shape

(92, 7)

In [ ]:
data_group = data.groupby(['place_id', 'place_name'])['city', 'category_1', 'category_2', 'category_3'].sum().reset_index()

data_group = {name: np.array(value) for name, value in data_group.items()}
data_group = tf.data.Dataset.from_tensor_slices(data_group)

places = data[['place_name']].drop_duplicates()
places = {name: np.array(value) for name, value in places.items()}
place = tf.data.Dataset.from_tensor_slices(places)

grouped_data = data_group.map(lambda x: {
    'place_id' : str(x['place_id']),
    'place_name' : x['place_name'],
    'city' : x['city'],
    'category_1' : x['category_1'],
    'category_2' : x['category_2'],
    'category_3' : x['category_3'],

})

place = place.map(lambda x: x['place_name'])

In [ ]:
unique_place = np.unique(np.concatenate(list(place.batch(1000))))
unique_city = np.unique(np.concatenate(list(grouped_data.batch(1_000).map(lambda x: x["city"]))))
unique_user_ids = np.unique(np.concatenate(list(grouped_data.batch(1_000).map(lambda x: x["category_1"]))))
unique_user_ids2 = np.unique(np.concatenate(list(grouped_data.batch(1_000).map(lambda x: x["category_2"]))))
unique_user_ids3 = np.unique(np.concatenate(list(grouped_data.batch(1_000).map(lambda x: x["category_3"]))))

In [ ]:
tf.random.set_seed(42)
shuffled = grouped_data.shuffle(100, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(70)
test = shuffled.skip(70).take(30)

In [ ]:
class RecommenderPlaceModel(tfrs.Model):
  def __init__(self, user_model, place_model):
    super().__init__()
    place_model = tf.keras.Sequential([
                                      tf.keras.layers.experimental.preprocessing.StringLookup(
                                          vocabulary=unique_place, mask_token=None),
                                      tf.keras.layers.Embedding(len(unique_place) + 1, embedding_dimension)
    ])
    self.place_model: tf.keras.Model = place_model

    user_model = tf.keras.Sequential([
                                      tf.keras.layers.experimental.preprocessing.StringLookup(
                                          vocabulary=unique_user_ids, mask_token=None),
                                      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),

    ])
    self.user_model: tf.keras.Model = user_model

    metrics = tfrs.metrics.FactorizedTopK(
      candidates=place.batch(10).map(place_model)
    )

    task = tfrs.tasks.Retrieval(metrics=metrics)
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    user_embeddings = self.user_model(features['category_1'], features['category_2'], features['category_3'])#, features['city'])
    place_embeddings = self.place_model(features['place_name'])
    return self.task(user_embeddings, place_embeddings)

In [ ]:
embedding_dimension = 96

place_model = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                      vocabulary=unique_place, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_place) + 1, embedding_dimension)
])

user_model = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                      vocabulary=unique_user_ids, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                          vocabulary=unique_city, mask_token=None),
                                      tf.keras.layers.Embedding(len(unique_city) +1, embedding_dimension)
])

model = RecommenderPlaceModel(user_model, place_model)

#model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.002))
cached_train = train.shuffle(10).batch(8).cache()
cached_test = test.batch(5).cache()

model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.5))
history = model.fit(cached_train, epochs=300)
#model.evaluate(cached_test, return_dict=True)


Epoch 1/300
9/9 [==============================] - 1s 62ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0143 - factorized_top_k/top_10_categorical_accuracy: 0.0429 - factorized_top_k/top_50_categorical_accuracy: 0.5286 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 15.4825 - regularization_loss: 0.0000e+00 - total_loss: 15.4825
Epoch 2/300
9/9 [==============================] - 1s 62ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0571 - factorized_top_k/top_10_categorical_accuracy: 0.1714 - factorized_top_k/top_50_categorical_accuracy: 0.7857 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 12.8467 - regularization_loss: 0.0000e+00 - total_loss: 12.8467
Epoch 3/300
9/9 [==============================] - 1s 61ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.1

In [ ]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 60ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.3636 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 5.8499 - regularization_loss: 0.0000e+00 - total_loss: 5.8499


{'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.0,
 'factorized_top_k/top_50_categorical_accuracy': 0.3636363744735718,
 'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'loss': 1.4165406227111816,
 'regularization_loss': 0,
 'total_loss': 1.4165406227111816}

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model, k=4)
index.index_from_dataset(
    place.batch(10).map(lambda title: (title, model.place_model(title)))
)

In [ ]:
_, name = index(tf.constant(["Ruang Terbuka", "Aktivitas Air", "Pantai"]))
print(f"Recommendations: {name[0]}")

Recommendations: [b'Tanjung Labora' b'Air Terjun Salo Merunge' b'Wisata Danau Raja'
 b'Salju Panas Dolok Tinggi Raja']


In [ ]:
#Save model
model.save_weights('Place Recommender.h5')
model.save_weights('content_model_weights', save_format='tf')

In [ ]:
tf.saved_model.save(index, "export")

In [ ]:
# Load it back; can also be done in TensorFlow Serving.
loaded = tf.saved_model.load("export")

# Pass a user id in, get top predicted vaccines
scores, name = loaded(["Relaxing", "Tempat Bersejarah", "Gunung"])

print(f"Recommendations: {name[0]}")

Recommendations: [b'Funland Mickey Holiday' b'Pusat Kerajinan Perahu Phinisi' b'Wakatobi'
 b'Jam Gadang']


In [ ]:
print(loaded.signatures['serving_default'].inputs)

[<tf.Tensor 'input_1:0' shape=(None,) dtype=string>, <tf.Tensor 'unknown:0' shape=() dtype=resource>, <tf.Tensor 'unknown_0:0' shape=() dtype=int64>, <tf.Tensor 'unknown_1:0' shape=() dtype=resource>, <tf.Tensor 'unknown_2:0' shape=() dtype=resource>, <tf.Tensor 'unknown_3:0' shape=() dtype=resource>]


In [ ]:
print(f"Recommendations: {scores[0]}")

Recommendations: [3.8007503 3.7986145 3.7980213 3.7961218]


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("export")
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

11476

In [ ]:
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

# get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)

# test the model
input_shape = input_details[0]['shape']
print(input_shape)

interpreter.set_tensor(input_details[0]['index'], np.array(["Tempat Bersejarah"]))
# interpreter.set_tensor(input_details[0]['index'], np.array(["Pfizer"]))

interpreter.invoke()

output_data = interpreter.get_tensor(output_details[1]['index'])
print(output_data[0])

[{'name': 'serving_default_input_1:0', 'index': 0, 'shape': array([1], dtype=int32), 'shape_signature': array([-1], dtype=int32), 'dtype': <class 'numpy.bytes_'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[1]
[b'Museum Tsunami' b'Masjid Raya Medan' b'Jam Gadang'
 b'Bukit Kasih Kanonang']
